                                                text
0  Pokédex\nLilian Besson\nJanuary 12, 2018\nCont...
                                                text
0   A Simple PDF File \n This is a small demonstr...


In [1]:
#this code converts text to a dataframe suitable for a neral network

import PyQt5
import PyPDF2
import pandas as pd
from PyQt5 import QtWidgets, QtCore
from PyQt5.QtWidgets import QFileDialog, QVBoxLayout, QTableView
from PyQt5.QtCore import QAbstractTableModel, Qt
from PIL import Image
from PyPDF2 import PdfReader


class PandasModel(QAbstractTableModel):
    def __init__(self, dataframe):
        QAbstractTableModel.__init__(self)
        self._dataframe = dataframe

    def rowCount(self, parent=None):
        return len(self._dataframe.values)

    def columnCount(self, parent=None):
        return self._dataframe.columns.size

    def data(self, index, role=Qt.DisplayRole):
        if index.isValid():
            if role == Qt.DisplayRole:
                return str(self._dataframe.iloc[index.row(), index.column()])
    def headerData(self, col, orientation, role):
        if orientation == Qt.Horizontal and role == Qt.DisplayRole:
            return self._dataframe.columns[col]
        return None

class PDFConverter(QtWidgets.QMainWindow):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        self.setGeometry(300, 300, 300, 300)
        self.setWindowTitle('PDF to Dataframe Converter')
        self.setFixedSize(500, 500)

        self.uploadButton = QtWidgets.QPushButton('Upload PDF', self)
        self.uploadButton.move(50, 50)
        self.uploadButton.clicked.connect(self.uploadPDF)

        self.progressBar = QtWidgets.QProgressBar(self)
        self.progressBar.setGeometry(50, 100, 200, 25)

        self.tableView = QTableView(self)
        self.tableView.move(50, 150)

        self.show()

    def uploadPDF(self):
        options = QFileDialog.Options()
        options |= QFileDialog.ReadOnly
        fileName, _ = QFileDialog.getOpenFileName(self,"QFileDialog.getOpenFileName()", "","PDF Files (*.pdf);;All Files (*)", options=options)
        if fileName:
            # Open the PDF file
            pdfFile = open(fileName, 'rb')
            pdfReader = PyPDF2.PdfFileReader(pdfFile)
            
            # Set the maximum value of the progress bar
            self.progressBar.setMaximum(pdfReader.numPages)
            self.progressBar.setValue(0)

            # Extract the text from the PDF
            text = []
            for page in range(pdfReader.numPages):
                text.append(pdfReader.getPage(page).extractText())
                self.progressBar.setValue(page)
                QtWidgets.QApplication.processEvents()

            # Convert the text into a dataframe
            self.df = pd.DataFrame(text, columns=['Page Text'])
            # Save the dataframe to a text file
            fileName, _ = QFileDialog.getSaveFileName(self, "Save File", "", "Text Files (*.txt);;All Files (*)")
            if fileName:
                self.df.to_csv(fileName, index=False)

if __name__ == '__main__':
    app = QtWidgets.QApplication([])
    pdf_converter = PDFConverter()
    app.exec_()

In [1]:
#this code converts images to a tensor/dataframe suitable for a neral network

import sys
from PyQt5 import QtWidgets, QtGui
from PIL import Image
import numpy as np
import torch

class ImageConverter(QtWidgets.QWidget):
    def __init__(self):
        super().__init__()

        # create a file picker button
        self.file_picker = QtWidgets.QPushButton('Select Image', self)
        self.file_picker.clicked.connect(self.select_image)

        # create a progress bar
        self.progress_bar = QtWidgets.QProgressBar(self)

        # create a 'Convert' button
        self.convert_button = QtWidgets.QPushButton('Convert', self)
        self.convert_button.clicked.connect(self.convert_image)
        self.convert_button.setEnabled(False)

        # create a layout and add the file picker and convert buttons
        layout = QtWidgets.QVBoxLayout(self)
        layout.addWidget(self.file_picker)
        layout.addWidget(self.progress_bar)
        layout.addWidget(self.convert_button)

    def select_image(self):
        options = QtWidgets.QFileDialog.Options()
        options |= QtWidgets.QFileDialog.ReadOnly
        file_name, _ = QtWidgets.QFileDialog.getOpenFileName(self, 'Select Image', '', 'Images (*.png *.xpm *.jpg *.bmp);;All Files (*)', options=options)
        if file_name:
            self.file_name = file_name
            self.convert_button.setEnabled(True)

    def convert_image(self):
        # Open image using PIL
        image = Image.open(self.file_name)
        # Convert image to a numpy array
        image_array = np.array(image)
        # Flatten the 3D array to a 1D array
        flat_array = image_array.flatten()
        # Save the 1D array to a text file
        options = QtWidgets.QFileDialog.Options()
        file_path, _ = QtWidgets.QFileDialog.getSaveFileName(self, "Save As", "", "Text Files (*.txt);;All Files (*)", options=options)
        if file_path:
            np.savetxt(file_path, flat_array)
            self.progress_bar.setValue(100)

if __name__ == '__main__':
    app = QtWidgets.QApplication(sys.argv)
    converter = ImageConverter()
    converter.show()
    sys.exit(app.exec_())

SystemExit: 0

C:\Users\Nikal\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3452: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [7]:
#this is an attempt to combine the two features
import PyPDF2
from PIL import Image
import pandas as pd
import io

class PDFConverter(QtWidgets.QMainWindow):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        self.setGeometry(300, 300, 300, 300)
        self.setWindowTitle('PDF to Dataframe Converter')
        self.setFixedSize(500, 500)

        self.uploadButton = QtWidgets.QPushButton('Upload PDF', self)
        self.uploadButton.move(50, 50)
        self.uploadButton.clicked.connect(self.uploadPDF)

        self.progressBar = QtWidgets.QProgressBar(self)
        self.progressBar.setGeometry(50, 100, 200, 25)

        self.tableView = QTableView(self)
        self.tableView.move(50, 150)

        self.show()

    def uploadPDF(self):
        options = QFileDialog.Options()
        options |= QFileDialog.ReadOnly
        fileName, _ = QFileDialog.getOpenFileName(self,"QFileDialog.getOpenFileName()", "","PDF Files (*.pdf);;All Files (*)", options=options)
        if fileName:
            self.df = extract_pdf_text_images(fileName)
            fileName, _ = QFileDialog.getSaveFileName(self, "Save File", "", "Text Files (*.txt);;All Files (*)")
            if fileName:
                self.df.to_csv(fileName, index=False)
                
    

NameError: name 'ImageConverter' is not defined